In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline


from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maloj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maloj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\maloj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [8]:
df=pd.read_csv('spam.csv')

In [9]:
print(df.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [10]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [11]:
df.rename(columns={'v1':'label', 'v2':'message'}, inplace=True)

In [12]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['label']=le.fit_transform(df['label'])

In [13]:
df.duplicated().sum()


403

In [14]:
df=df.drop_duplicates(keep='first')


In [15]:
from nltk.stem.porter import PorterStemmer
import string
ps=PorterStemmer()

In [16]:
def  transform_text(message):
    message=message.lower()
    message=nltk.word_tokenize(message)
    y=[]
    for i in message:
        if i.isalnum():
            y.append(i)
    message=y[:]
    y.clear()
    
    for i in message:
            if i not in stopwords.words('english') and i not in string.punctuation:
                y.append(ps.stem(i))
    return " ".join(y)

In [19]:
df['transformed_message']=df['message'].apply(transform_text)
df.head()

,label,message,transformed_message
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tfidf=TfidfVectorizer(max_features=500)

In [22]:
X=tfidf.fit_transform(df['transformed_message']).toarray()
y=df['label'].values

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.20,random_state=2)


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier



In [25]:
svc= SVC(kernel="sigmoid",gamma=1.0)
knc=KNeighborsClassifier()
mnb=MultinomialNB()
dtc=DecisionTreeClassifier(max_depth=5)
rfc=RandomForestClassifier(n_estimators=50,random_state=2)
lrc=LogisticRegression(solver='liblinear',penalty='l1')
abc=AdaBoostClassifier(n_estimators=50,random_state=2)
bgc=BaggingClassifier(n_estimators=50,random_state=2)


In [32]:
clfs={
    'SVC':svc,
    'KNC':knc,
    'MNB':mnb,
    'DTC':dtc,
    'RFC':rfc,
    'LRC':lrc,
    'ABC':abc,
    'BGC':bgc
}

In [33]:
from sklearn.metrics import accuracy_score,precision_score
def train_classifier(clfs,X_train,y_train,X_test,y_test):
    clfs.fit(X_train,y_train)
    y_pred=clfs.predict(X_test)
    acccurecy=accuracy_score(y_test,y_pred)
    precision=precision_score(y_test,y_pred)
    return acccurecy,precision

In [34]:
accuracy_scores=[]
pression_scores=[]
for name,clfs in clfs.items():
    current_accuracy,current_precision=train_classifier(clfs,X_train,y_train,X_test,y_test)
    print()
    print("For ",name)
    print("Accuracy :",current_accuracy)
    print("Precision :",current_precision)
    accuracy_scores.append(current_accuracy)
    pression_scores.append(current_precision)   


For  SVC
Accuracy : 0.9661508704061895
Precision : 0.9327731092436975

For  KNC
Accuracy : 0.9274661508704062
Precision : 1.0

For  MNB
Accuracy : 0.9709864603481625
Precision : 0.9655172413793104

For  DTC
Accuracy : 0.9361702127659575
Precision : 0.9

For  RFC
Accuracy : 0.971953578336557
Precision : 0.943089430894309

For  LRC
Accuracy : 0.9622823984526112
Precision : 0.9541284403669725


d:\Users\maloj\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



For  ABC
Accuracy : 0.9613152804642167
Precision : 0.9375

For  BGC
Accuracy : 0.965183752417795
Precision : 0.9180327868852459


In [35]:
def check_spam(message):
    cleaned = transform_text(message)
    vector = tfidf.transform([cleaned])
    result = rfc.predict(vector)
    
    return "SPAM" if result[0] == 1 else "HAM"


In [ ]:
check_spam("Hey, are we meeting today?")



'HAM'

In [38]:
check_spam("Free prize!!! Click win now")

'SPAM'